In [1]:
import torch 
import torch.optim as optim

import matplotlib.pyplot as plt

import glob

import numpy as np

from utils.tustinNet import TustinNet
from utils.loadData import LoadData
from utils.tustinLoss import Loss

In [2]:
batch_size = 32
seq_len = 50
step_size = 5

trainPath = "..\\..\\data\\train\\"
trainPaths = glob.glob(trainPath + "*.csv")

_, trainloader, theta_scale, alpha_scale = LoadData(files = trainPaths, seq_len = seq_len, step_size = step_size, batch_size = batch_size, truncation=False)


valPath = "..\\..\\data\\validation\\"
valPaths = glob.glob(valPath + "*.csv")
_, valloader, _, _ = LoadData(files = valPaths, seq_len = seq_len, step_size = step_size, batch_size = batch_size, truncation=False)

In [ ]:
model = TustinNet(input_size = 7, layer_size = 100, output_size = 2, batch_size = batch_size, theta_scale = theta_scale, alpha_scale = alpha_scale)

optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

In [ ]:
numEpochs = 120
trainLossList = []
valLossList = []
lrList = []
bestLoss = None
for epoch in range(numEpochs):
    trainLoss = 0.0
    valLoss = 0.0
    for _, (u_batch, y_batch) in enumerate(trainloader):
        # Select initial states
        hidden0 = y_batch[:,0,:]          # shape (batch_size, 1, 4)

        # Zero the gradients
        model.zero_grad()

        # Forward pass
        y_pred, _ = model(u_batch, hidden0)

        # Evaluate the loss
        loss = Loss(y_pred, y_batch)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()
        
        # Sum loss of batch
        trainLoss += loss.item()

    # Save weights of best model
    if bestLoss == None:
        bestLoss = trainLoss
        torch.save(model.state_dict(), "models\\standard.pt")
    elif trainLoss < bestLoss:
        bestLoss = trainLoss
        torch.save(model.state_dict(), "models\\standard.pt")

    # Evaluate validation loss
    for _, (u_batch, y_batch) in enumerate(valloader):
        hidden0 = y_batch[:,0,:]
        model.eval()
        with torch.no_grad():
            y_pred, _ = model(u_batch, hidden0)
            loss = Loss(y_pred, y_batch)
            valLoss += loss.item()

    # Decrease learning rate
    scheduler.step(valLoss)
    
    # Add values to lists
    lrList.append(scheduler._last_lr)
    trainLossList.append(trainLoss)
    valLossList.append(valLoss)

In [ ]:
plt.semilogy(trainLossList)
plt.semilogy(valLossList)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

trainLossList[-1], min(trainLossList), min(valLossList)

In [ ]:
plt.plot(lrList)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.show()